In [2]:
# My created .py files for modularization
import env
import os
import wrangle as w
# Ignore Warning
import warnings
warnings.filterwarnings("ignore")
# Array and Dataframes
import numpy as np
import pandas as pd
# Imputer
from sklearn.impute import SimpleImputer
# Evaluation: Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Evaluation: Statistical Analysis
from scipy import stats
# Modeling: Scaling
from sklearn.preprocessing import QuantileTransformer
# Modeling
from sklearn.model_selection import GridSearchCV
# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [3]:
# delimit columns showed for ease
pd.set_option('display.max_columns', None)

In [4]:
# defining dataframe from wrangle
zillow = w.zillow()

File exists, pulling from system.


## Data Previews & Insight

In [5]:
# sample
zillow.sample(5)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,county
745468,3,2.0,2141.0,303233.0,1940,4235.57,LA
1820531,4,2.0,1560.0,105528.0,1955,1333.90,LA
173643,4,3.0,2834.0,694606.0,1936,8453.57,LA
780133,2,1.0,810.0,248612.0,1949,3475.14,LA
991575,5,3.0,2622.0,469145.0,1967,5916.86,LA


In [15]:
# explore columns: nunique, unique, isna, dtype
x = w.summarize(zillow)

print(zillow.dtypes)
x

bedroomcnt                        int64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                         int64
taxamount                       float64
county                           object
dtype: object


,Column Name,Number of Unique Values,Unique Values,Number of Null Values,dtype
6,county,3,"[LA, Orange, Venture]",0,object
0,bedroomcnt,19,"[4, 3, 0, 5, 2, 1, 6, 7, 8, 9, 25, 10, 11, 14,...",0,int64
1,bathroomcnt,38,"[2.0, 4.0, 0.0, 1.0, 2.5, 3.5, 3.0, 5.5, 4.5, ...",0,float64
4,yearbuilt,153,"[2005, 2011, 1926, 1972, 1973, 1960, 1950, 195...",0,int64
2,calculatedfinishedsquarefeet,10557,"[3633.0, 1620.0, 2077.0, 1200.0, 171.0, 203.0,...",0,float64
3,taxvaluedollarcnt,588178,"[296425.0, 847770.0, 646760.0, 5328.0, 6920.0,...",0,float64
5,taxamount,915868,"[6941.39, 10244.94, 7924.68, 91.6, 255.17, 163...",0,float64


In [40]:
#checking for values with no decimals but floats
    # cols = ['bedroomcnt','taxvaluedollarcnt','calculatedfinishedsquarefeet','yearbuilt']
    # for col in cols:
    #     if zillow[col][zillow[col] < 1]:
    #         print(f'{col} no decimals.')
    #     else:
    #         print(f'{col} does have decimals.')
    

In [7]:
# info
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    int64  
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     int64  
 5   taxamount                     float64
 6   county                        object 
dtypes: float64(4), int64(2), object(1)
memory usage: 130.6+ MB


In [8]:
# describe
zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount
count,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06
mean,3.301419e+00,2.240352e+00,1.863194e+03,4.606416e+05,1.960968e+03,5.616711e+03
std,9.325998e-01,9.905489e-01,1.221754e+03,6.771576e+05,2.215056e+01,7.814563e+03
min,0.000000e+00,0.000000e+00,1.000000e+00,2.200000e+01,1.801000e+03,6.340000e+00
25%,3.000000e+00,2.000000e+00,1.258000e+03,1.891660e+05,1.949000e+03,2.540850e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.282960e+05,1.958000e+03,4.111470e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.346060e+05,1.976000e+03,6.411930e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.018846e+07,2.016000e+03,1.078102e+06


In [9]:
# checking amount of outliers
zillow[zillow.yearbuilt.between(1800,1900)]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,county
207,3,1.0,1319.0,114727.0,1890,1270.80,Venture
3549,2,1.0,840.0,393000.0,1885,4850.66,LA
3551,3,1.0,1674.0,418000.0,1887,5197.58,LA
3917,4,2.0,1488.0,226336.0,1900,2757.69,LA
3990,3,1.0,1118.0,265257.0,1900,3195.31,LA
...,...,...,...,...,...,...,...
2147922,2,2.0,1276.0,85584.0,1894,1459.52,LA
2148015,2,1.0,1270.0,284667.0,1895,3486.72,LA
2148041,3,2.0,1134.0,28737.0,1900,378.50,LA
2148840,3,1.0,1469.0,147329.0,1885,1974.96,LA


In [10]:
yearmean = zillow['yearbuilt'].mean().round(0)
zillow['yearbuilt'].fillna(yearmean,inplace=True)

In [11]:
# drop all values with missing bedroomcnt values
zillow[zillow.bedroomcnt.isnull()].dropna(inplace=True)

In [12]:
zillow.taxamount.max()

1078101.87